Мини-поисковик по базе текстов

План работы прост
Скачать текста
Проиндексировать
Поисковик по индексам

Для скачивания возьмем несколько классических произведений, война и мир 4 тома

In [9]:
raw=[
    'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0040.shtml',
    'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0050.shtml',
    'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0060.shtml',
    'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0070.shtml'
]

import hashlib
hash_raw={}
for i in raw:
    hash_object = hashlib.md5(i.encode())
    hash_raw[hash_object.hexdigest()]=i

import requests
for i in hash_raw:
    url=hash_raw[i]
    r = requests.get(url)
    text=r.content.decode(encoding='cp1251', errors="ignore")
    with open('raw/'+i+'.html', "w") as text_file:
        text_file.write(text)
        


Скачав текста можно заняться индексацией, будет индексировать максимально просто по словам которые больше 2х символов
Убрав все спец символы перед этим 

In [22]:
import re

hash_words={}
for i in hash_raw:
    with open('raw/'+i+'.html', "r") as text_file:
        text=text_file.read()
        text=re.sub('[^0-9a-zA-Zа-яА-Я]',' ',text)
        words=text.split()
        words_hash={}
        for j in words:
            if len(j)<2:continue
            if j in words_hash:
                words_hash[j]=words_hash[j]+1
            else:
                words_hash[j]=1
        hash_words[i]=words_hash
        

Получив списко слов и число их вхождений теперь нужно сохранить их в какую либо структуру где не страница будет вести к слову а слово к странице
Дерево из хеш мап выглядит как вполне подходящее для этих целей

In [26]:
class Index:
    def __init__(self):
        self.tree={}
        self.words={}
    

    def insert(self,word:str,data):
        if len(word)>1:
            char=word[0]
            if char not in self.tree:
                self.tree[char]=Index()
            self.tree[char].insert(word[1:],data)
            return
        
        if word in self.words:
            self.words[word]=self.words[word]+data
        else:
            self.words[word]=data
        return
    
    def search(self,word):
        if len(word)>1:
            char=word[0]
            if char not in self.tree:
                return None
            return self.tree[char].search(word[1:])
        if word in self.words:
            return self.words[word]
        return None

In [27]:
index=Index()
for i in hash_words:
    for j in hash_words[i]:
        index.insert(j,[i,hash_words[i][j]])

Теперь имя на руках индексы всех вхождений кажого слова и того куда и в каком количестве они входят нужна функция поисковика, которая сможет объеденить результаты по множеству слов котоыре подаются на вход поисковой функции

In [97]:
def search(hash_raw,index,to_search:str, true_to_or:bool=True):
    text=re.sub('[^0-9a-zA-Zа-яА-Я]',' ',to_search)
    words=text.split()

    results=[]
    for i in words:
        res=index.search(i)
        if res is None and true_to_or is False:return []
        if res is None: continue
        results.append(res)
    results_all=[]
    results_all_uniq={}
    if true_to_or is True:
        
        for i in results:
            for j in range(0,len(i),2):
                results_all.append([i[j],i[j+1]])
        
        results_all_uniq={}
        for i in results_all:
            if i[0] in results_all_uniq:
                results_all_uniq[i[0]]=results_all_uniq[i[0]]+i[1]
            else:
                results_all_uniq[i[0]]=i[1]
        
        
    else:
        results_all_uniq={}
        first=True
        for i in results:
            this_result={}
            for j in range(0,len(i),2):
                key=i[j]
                value=i[j+1]

                if first is True:
                    results_all_uniq[key]=value
                else:
                    this_result[key]=value
            
            
            if first is True:
                first=False
            else:
                for k in this_result:
                    if k in results_all_uniq:
                        results_all_uniq[k]=results_all_uniq[k]+this_result[k]
                to_drop=[]
                for k in results_all_uniq:
                    if k not in this_result:
                        to_drop.append(k)
                for j in to_drop:
                    results_all_uniq.pop(j)       
    results_all=[]
    for k in results_all_uniq: 
        results_all.append([k,results_all_uniq[k]])
    results_all.sort(key = lambda x: x[1],reverse=True)

    for i in results_all:
        i[0]=hash_raw[i[0]]
    return results_all



In [100]:
result=search(hash_raw,index,'что или ',False)
for i in result:
    print(i)

['http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0060.shtml', 2393]
['http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0070.shtml', 2269]
['http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0050.shtml', 2172]
['http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0040.shtml', 1778]
